In [1]:
import wandb


import pandas as pd 
import pyVCT 
import json
from utils.draw import make_image 
from utils.auxiliary import *

data = pd.read_csv('data.csv')

In [2]:
def train(config=None):
    with wandb.init(config=config):
        config = wandb.config

        conf_dict = dict(config)

        cm_data, fb_data = get_exp_distributions(data, config['CELL_DAY_TYPE'])
        
        del conf_dict['CELL_DAY_TYPE']

        types, c_tags, fibers, contacts = pyVCT.py_cpmfem(17, 68, 0.5, 0.0025, 1, 1, 'monolayer_without_fiber', 3001,
                                                         **conf_dict)
        CM, FB = make_df(c_tags, types)

        metrics = compute_metrics(CM, FB, types, c_tags, fibers, contacts, cm_data, fb_data)
        
        wandb.log(metrics)  

In [ ]:
with open('sweeps/d20_sweeps/jcmmd_jfbmd.json', 'r') as fp:
    sweep_config = json.load(fp)
    
sweep_id = wandb.sweep(sweep_config, project="potts-optimization")

In [ ]:
wandb.agent(project="potts-optimization", sweep_id='o513hnvh', function=train)